In [1]:

# import zipfile

# # Ruta del archivo ZIP
# zip_file_path = '/content/tesis-main.zip'

# # Ruta donde quieres extraer los archivos
# extract_to_path = '/content/'

# # Descomprimir el archivo
# with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
#     zip_ref.extractall(extract_to_path)

# print("Archivos extraídos con éxito.")

In [2]:
# import sys
# sys.path.append('/content/tesis-main/')

from dotenv import load_dotenv

load_dotenv()

True

In [3]:

# import drive as drive_downloader
# drive_downloader.download_files('14_UUqgoWDPPNwXvdFjDoJomHTu5l85l6')

* Carga el video utilizando cv2.VideoCapture.
* Extrae los frames del video uno por uno.
* Preprocesa cada frame para que sean compatibles con el modelo de PyTorch (tamaño, normalización, etc.).
* Usa un modelo preentrenado de PyTorch (como ResNet) para extraer características de cada frame.
* Aplica un algoritmo de clustering (como K-Means) a las características extraídas.
* Asigna cada frame al cluster correspondiente.
* Visualiza los resultados mostrando los frames y su cluster correspondiente.

In [4]:
import torch
from sklearn.cluster import AgglomerativeClustering
import warnings
from preprocessing.transforms import BASELINE
from clustering.metrics import eval_massive_cluster
from experiment_framework import experiment
from validation import VALIDATION_DATASET
from utils import load_dataset
warnings.filterwarnings('ignore')
import wandb

torch.manual_seed(47)
import random
random.seed(47)
import numpy as np
np.random.seed(47)

if torch.cuda.is_available():
 dev = "cuda:0"
else:
 dev = "mps"


device = torch.device(dev)
epochs = 10
steps=1
clustering_eval_function = lambda cluster_labels, tags: eval_massive_cluster(cluster_labels, tags, steps, epochs)

dataset_loader = load_dataset(VALIDATION_DATASET.keys())

# wandb.login()

In [5]:
from properties import MODELS_ENUM, PREPROCESSORS, ALGORITHM, GROUPER_FUNCTIONS

DEFAULT_PARAMS = {
      "dataset": dataset_loader,
      "evaluation_function": clustering_eval_function,
      "show": True,
      "device": device
   }

EXPERIMENT_DEFINITIONS_OLD= [
    {
        'name': 'YOLO',
        'model': {
          'name': "YOLOV8",
          'params': ['best.pt']
        },
        'algorithm': "AGGLOMERATIVE",
        'params': { "n_clusters":None, "distance_threshold": 17},
        'grouper_function': "CLUSTERING_FUNCTION",
        'preprocessing': "YOLO_BASELINE",
    },
    {
        'name': 'REsnet',
        'model': {
          'name': "RESNET18",
          'params': [device]
        },
        'algorithm': "SIMILARITY",
        'params': {"threshold": 0.99},
        'grouper_function': "CLUSTERING_FUNCTION",
        'preprocessing': "YOLO_BASELINE",
    },
    {
        'name': 'Siamese',
        'model': {
          'name': "SIAMESE_RESNET18",
          'params': [device, 'siameseResnet18EuclideanFull.pt']
        },
        'algorithm': "SIMILARITY",
        'params': {"threshold":1, "distance": "euclidean"},
        'grouper_function': "CLUSTERING_FUNCTION",
        'preprocessing': "BASELINE",
    },
    {
        'name': 'Encoder yolo mask clustering',
        'model': {
          'name': "Encoder",
          'params': ['best.pt', 'autoencoderv5.pt', device]
        },
        'algorithm': "SIMILARITY",
        'params': {"threshold": 0.8},
        'grouper_function': "CLUSTERING_FUNCTION",
        'preprocessing': "YOLO_BASELINE",
    },
    {
        'name': 'YOLORES distance',
        'model': {
          'name': "YOLORES",
          'params': [device, 'best.pt']
        },
        'algorithm': "AGGLOMERATIVE",
        'params': { "n_clusters":None, "distance_threshold": 17},
        'grouper_function': "CLUSTERING_FUNCTION",
        'preprocessing': "YOLO_BASELINE",
    },
]


In [6]:
from itertools import product
import copy
import json


AGGLOMERATIVE_PARAMS = {
            'distance_threshold':[0.3,0.5,1,2,4,5,7,10,12],
            'algorithm':["AGGLOMERATIVE"]
        }

AGGLOMERATIVE_TEMPLATE={
          'model': {
            'name': '{model_name}',
            'params': '{model_params}'
          },
          'algorithm': '{algorithm}',
          'params': { "n_clusters":None, "distance_threshold": "{distance_threshold}"},
          'grouper_function': "CLUSTERING_FUNCTION",
          'preprocessing': '{preprocessing}',
      }



SIMILARITY_PARAMS={
            "threshold":[.3,.5,.8,.9,.99,.999,.9999,1,1.1,1.2,1.5], 
            "distance": ["euclidean","cosine"],
            'algorithm':["SIMILARITY"]
        }
SIMILARITY_TEMPLATE={
          'model': {
            'name': '{model_name}',
            'params': '{model_params}'
          },
          'algorithm': '{algorithm}',
          'params': { "threshold":'{threshold}', "distance":'{distance}' },
          'grouper_function': "CLUSTERING_FUNCTION",
          'preprocessing': '{preprocessing}',
      }

EXPERIMENTS_DEFINITIONS= [

       {
        'embeddings_params':{
            'preprocessing':["YOLO_BASELINE"],
            'model_name':['YOLORES'],
            'model_params':[[device,'best_rico.pt'],[device,'best.pt']]
        },
        'other_params':AGGLOMERATIVE_PARAMS,

      'template':AGGLOMERATIVE_TEMPLATE,
      'name': "YOLO_RESNET Agglomerative"
    },  
        {
        'embeddings_params':{
            'preprocessing':["YOLO_BASELINE"],
            'model_name':["YOLORES"],
            'model_params':[[device,'best_rico.pt'],[device,'best.pt']]
        },
        'other_params':SIMILARITY_PARAMS,
      'template':SIMILARITY_TEMPLATE,
      'name': "YOLO_RESNET Similarity"
    }

#  {
#         'embeddings_params':{
#             'preprocessing':["YOLO_BASELINE"],
#             'model_name':['Encoder'],
#             'model_params':[['best.pt', 'autoencoderv5.pt', device]]

#         },
#         'other_params':AGGLOMERATIVE_PARAMS,

#       'template':AGGLOMERATIVE_TEMPLATE,
#       'name': "Encoder yolo mask clustering Agglomerative"
#   },
#  {
#         'embeddings_params':{
#             'preprocessing':["YOLO_BASELINE"],
#             'model_name':['Encoder'],
#             'model_params':[['best.pt', 'autoencoderv5.pt', device]]

#         },
#         'other_params':SIMILARITY_PARAMS,

#       'template':SIMILARITY_TEMPLATE,
#       'name': "Encoder yolo mask clustering  Similarity"
#     },



# {
#         'embeddings_params':{
#             'preprocessing':["BASELINE"],
#             'model_name':['SIAMESE_RESNET18'],
#             'model_params':[[device, 'siameseResnet18EuclideanFull.pt']]

#         },
#         'other_params':AGGLOMERATIVE_PARAMS,

#       'template':AGGLOMERATIVE_TEMPLATE,
#       'name': "Siamese Resnet Agglomerative"
#   },
#  {
#         'embeddings_params':{
#             'preprocessing':["BASELINE"],
#             'model_name':['SIAMESE_RESNET18'],
#             'model_params':[[device,'siameseResnet18EuclideanFull.pt' ]]

#         },
#         'other_params':SIMILARITY_PARAMS,

#       'template':SIMILARITY_TEMPLATE,
#       'name': "Siamese Resnet Similarity"
#     },







#  {
#         'embeddings_params':{
#             'preprocessing':["BASELINE"],
#             'model_name':['RESNET18'],
#             'model_params':[[device]]

#         },
#         'other_params':AGGLOMERATIVE_PARAMS,

#       'template':AGGLOMERATIVE_TEMPLATE,
#       'name': "Resnet Agglomerative"
#   },
#  {
#         'embeddings_params':{
#             'preprocessing':["BASELINE"],
#             'model_name':['RESNET18'],
#             'model_params':[[device]]

#         },
#         'other_params':SIMILARITY_PARAMS,

#       'template':SIMILARITY_TEMPLATE,
#       'name': "Resnet Similarity"
#     },
    #     {
    #     'embeddings_params':{
    #         'preprocessing':["YOLO_BASELINE"],
    #         'model_name':['YOLOV8'],
    #         'model_params':[['best_rico.pt'],['best.pt']]
    #     },
    #     'other_params':AGGLOMERATIVE_PARAMS,

    #   'template':AGGLOMERATIVE_TEMPLATE,
    #   'name': "YOLO Agglomerative"
    # },  
    #     {
    #     'embeddings_params':{
    #         'preprocessing':["YOLO_BASELINE"],
    #         'model_name':["YOLOV8"],
    #         'model_params':[['best_rico.pt'],['best.pt']]
    #     },
    #     'other_params':SIMILARITY_PARAMS,
    #   'template':SIMILARITY_TEMPLATE,
    #   'name': "YOLO Similarity"
    # }
]
def get_experiment_definitions(exps):
  exps_emb = exps['embeddings_params'] 
  params  = {**exps_emb, **exps['other_params']}
  config = exps['template']
  name = exps['name']

  
  # Función recursiva para reemplazar valores en el JSON
  def replace_values(d, key, value_to_replace):
      if isinstance(d, dict):
          for k, v in d.items():
              if isinstance(v, dict) or isinstance(v, list):
                  replace_values(v, key, value_to_replace)
              elif isinstance(v, str) and v == "{" + key + "}":
                  d[k] = value_to_replace
      elif isinstance(d, list):
          for i, item in enumerate(d):
              if isinstance(item, dict) or isinstance(item, list):
                  replace_values(item, key, value_to_replace)
              elif isinstance(item, str) and item == "{" + key + "}":
                  d[i] = value_to_replace

  # Generar todas las combinaciones de los valores en params
  combinations = list(product(*params.values()))

  configs_modified = []

  for combination in combinations:
      # Crear una copia profunda del JSON original para cada combinación
      config_modified = copy.deepcopy(config)
      params_log =  zip(params.keys(), combination)
      name_components = [name]
      embeddings_hyperparams = []
      for key, value in params_log:
          replace_values(config_modified, key, value)
          name_components.append(f"{key}={value}")
          if key in exps_emb.keys():
              embeddings_hyperparams.append(f"{key}={value}")
      config_modified['name'] = ', '.join(name_components)
      config_modified['hyperparams'] = ', '.join(embeddings_hyperparams)
      configs_modified.append(config_modified)

  # Imprimir los JSON modificados para verificar
  # for config in configs_modified:
  #     print(json.dumps(config, indent=4))
  return configs_modified

for exps_definition in EXPERIMENTS_DEFINITIONS:
  EXPERIMENT_DEFINITIONS = get_experiment_definitions(exps_definition)
  for exp_definition in EXPERIMENT_DEFINITIONS:
    run = wandb.init(project="testing-ort3",group=exps_definition['name'], entity="hortensias")
    params = DEFAULT_PARAMS.copy()
    model_definition = MODELS_ENUM[exp_definition["model"]["name"]]
    model_params = exp_definition["model"]["params"]
    model = model_definition.get_instance(*model_params)
    params["model"] = model
    params["preprocessing"] = PREPROCESSORS[exp_definition["preprocessing"]]
    algorithm = ALGORITHM[exp_definition["algorithm"]](**exp_definition["params"])
    grouper_function = GROUPER_FUNCTIONS[exp_definition["grouper_function"]](algorithm)
    params["grouper_function"] = grouper_function
    params["name"] = exp_definition["name"]
    params["dataset"] = dataset_loader()
    params['hyperparams']=exp_definition["hyperparams"]
    result = experiment(**params)
    print(model.get_short_name(params['hyperparams']))
    run.log({
      "PCA_FILE":model.get_short_name(params['hyperparams']),
      "hyperparams": exp_definition['name'],
      "Precision": result.precision.mean,
      "Accuracy": result.accuracy.mean,
      "F1": result.f1.mean,
      "Recall": result.recall.mean,
      "Precision_STD": result.precision.std,
      "Accuracy_STD": result.accuracy.std,
      "F1_STD": result.f1.std,
      "Recall_STD": result.recall.std,
    })
    run.finish()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: piodanilod (hortensias). Use `wandb login --relogin` to force relogin


RuntimeError: Invalid device string: 'best_rico.pt'